1. Wstęp
    Problemem, którym się zajmowałem było Sudoku. Problem polega na wypełnieniu pustych pól w planszy tak, aby każdy wiersz, kolumna i kwadrat 3x3 zawierał wszystkie cyfry od 1 do 9. W tym celu wykorzystałem algorytm genetyczny. Algorytm może zarówno, generować rozwiązania dla podanych plansz, jak i generować poprawną plansze Sudoku, bez żadnego wcześniejszego inputu.
    Algorytm działa następująco:

In [2]:
import numpy as np
import pygad
import math

board_to_solve = np.array([
    [0, 2, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 3, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [9, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 0, 0]
])
sudoku_width = 6
sudoku_size = 36


def fitness_func(solution, solution_idx):
    # Reshape solution into a 9x9 sudoku grid
    sudoku = np.reshape(solution, (sudoku_width, sudoku_width))

    for i in range(sudoku_width):
        for j in range(sudoku_width):
            if board_to_solve[i][j] != 0:
                sudoku[i][j] = board_to_solve[i][j]

    # Calculate the fitness of the solution
    fitness = 1

    for i in range(sudoku_width):
        # Check the row
        fitness += sudoku_width - len(np.unique(sudoku[i, :]))

        # Check the column
        fitness += sudoku_width - len(np.unique(sudoku[:, i]))

    # Check the 3x3 subgrids
    for i in range(sudoku_width // 3):
        for j in range(sudoku_width // 3):
            fitness += 9 - len(np.unique(sudoku[i * 3:(i + 1) * 3, j * 3:(j + 1) * 3]))

    return 1 / fitness

Algorytm działa na zasadzie oceny planszy wygenerowanej, licząc jak dobre są poszczególne wiersze, kolumny i kwadraty 3x3. Liczy on to na podstawie tego ile jest unikalnych wartości, czyli tak naprawdę ile jest powtórzeń. Na początku dodajemy do genu wcześniej podane pola z tablicy do rozwiązania. Następnie liczymy fitness, który jest odwrotnością sumy wszystkich powtórzeń. Im mniej powtórzeń tym lepsza plansza.
Dla idealnego rozwiązania fitness wynosi 1. Ponieważ na początku ma on 1 i jeśli nie ma zadnych powtórzeń to dodajemy 0, więc fitness pozostaje 1.

Następnie ustawiłem parametry algorytmu, takie jak liczba pokoleń, rodzaj selekcji rodziców, rodzaj krzyżowania, rodzaj mutacji, liczba rodziców, które zostaną wybrane do krzyżowania, liczba rodziców, które zostaną wybrane do krzyżowania, liczba osobników w populacji, liczba genów w osobniku. Podane wartości oddawały najlepsze wyniki. Liczba generacji jest ustawiona na bardzo wysoką, ponieważ o ile w wypadku 3x3 lub 6x6 algorytm działa szybko, to przy 9x9 jest on znacznie bardziej czasochłonny. Problemem jest również, to że w przypadku 9x9, algorytm bez planszy generuje własną poprawną, a jeśli damy mu do rozwiązania plansze, to potrzebuje zdecydowanie więcej iteracji i wykonuje się bardzo długo.

In [3]:
num_generations = 5000
parent_selection_number = 20
keep_parents = 8
parent_selection_type = "sss"
mutation_percent_genes = 1 / sudoku_size * 100
sol_per_pop = 200

initial_population = []
for i in range(sol_per_pop):
    initial_population.append(np.random.randint(1, 10, size=sudoku_size))

ga_instance = pygad.GA(initial_population=initial_population,
                       num_generations=num_generations,
                       num_parents_mating=parent_selection_number,
                       keep_parents=keep_parents,
                       parent_selection_type=parent_selection_type,
                       mutation_percent_genes=mutation_percent_genes,
                       fitness_func=fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=sudoku_size,
                       gene_type=int,
                       crossover_type="two_points",
                       mutation_type="random",
                       gene_space=[1, 2, 3, 4, 5, 6, 7, 8, 9],
                       stop_criteria="reach_1.0"
                       )

ga_instance.run()

best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()

solution_array = np.reshape(best_solution, (sudoku_width, sudoku_width))

print("Solution : \n", solution_array, best_solution_fitness)

Solution : 
 [[6 2 9 1 7 4]
 [7 8 3 6 2 5]
 [4 5 1 3 8 9]
 [5 4 2 7 1 8]
 [9 1 8 4 3 6]
 [3 6 7 9 5 2]] 1.0


Przykładowe inputy:
    3x3:

    1.[[0, 2, 0],
    [0, 0, 0],
    [0, 0, 0]],
    output:
    [[7 2 9]
     [1 6 4]
     [5 3 8]]


    2.[[1, 7, 0],
    [0, 0, 0],
    [0, 0, 3]],
    output:
    [[1 7 4]
     [9 8 2]
     [6 5 3]]

    3.[[0, 6, 0],
    [0, 0, 0],
    [1, 0, 0]]
    output:
    [[9 6 5]
     [7 8 3]
     [1 4 2]]

    6x6:

    1.[[0, 2, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 3, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [9, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 0, 0]]
    output:
    [[8 2 9 7 5 1]
     [1 5 3 4 2 6]
     [6 7 4 3 9 8]
     [5 1 2 6 3 7]
     [9 8 6 2 4 5]
     [3 4 7 1 8 9]]


    2.[[0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 2, 0, 5, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 0, 0]]
    output:
    [[6 1 3 4 2 9]
     [5 9 8 1 7 6]
     [7 2 4 5 3 8]
     [3 8 6 9 4 1]
     [1 4 9 2 5 7]
     [2 5 7 8 6 3]]

    3.[[0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [3, 0, 0, 0, 0, 2],
    [2, 0, 0, 0, 0, 0],
    [0, 0, 8, 0, 0, 0]]
    output:
    [[9 6 3 2 7 5]
     [8 2 1 9 4 6]
     [5 7 4 1 8 3]
     [3 4 9 5 6 2]
     [2 1 7 8 3 4]
     [6 5 8 7 9 1]]

    9x9:

    1.[[0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 8, 5],
    [0, 0, 1, 0, 2, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 7, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 1, 0, 0],
    [0, 9, 0, 0, 0, 0, 0, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 7, 3],
    [0, 0, 2, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 9]]
    output:
    [[9 8 7 6 5 4 3 2 8]
    [2 4 6 9 7 3 0 8 5]
    [3 5 1 8 2 6 9 4 7]
    [1 2 3 5 8 7 4 9 6]
    [7 2 4 3 9 2 1 5 8]
    [8 9 5 1 6 0 7 3 2]
    [5 1 9 4 8 0 2 7 3]
    [4 7 2 0 1 9 5 6 8]
    [6 3 8 7 4 5 0 7 9]]

    2.[[0, 0, 0, 0, 4, 0, 2, 0, 0],
    [0, 0, 6, 5, 0, 8, 0, 0, 0],
    [5, 0, 0, 0, 0, 1, 0, 0, 0],
    [1, 0, 0, 8, 6, 0, 0, 0, 3],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [3, 0, 0, 0, 5, 7, 0, 0, 1],
    [0, 0, 0, 9, 0, 0, 0, 0, 2],
    [0, 0, 0, 7, 0, 0, 9, 6, 0],
    [0, 0, 1, 0, 3, 0, 0, 0, 0]]
    output:
    [[7 2, 3 4 5 6 9 1 8]
    [4 1 9 2 7 3 6 8 5]
    [6 8 5 1 2 9 4 3 7]
    [3 6 8 5 9 7 2 4 1]
    [2 7 4 8 6 1 5 9 3]
    [1 9 5 3 4 2 8 7 6]
    [5 4 6 9 8 1 7 2 3]
    [8 3 2 7 1 5 9 6 4]
    [9 5 1 6 3 4 8 7 2]]

    3.[[7, 9, 0, 0, 0, 3, 0, 0, 0],
     [8, 0, 0, 0, 0, 7, 2, 0, 0],
     [0, 0, 0, 0, 0, 9, 0, 8, 0],
     [0, 0, 0, 0, 0, 4, 5, 9, 0],
     [0, 5, 0, 0, 6, 0, 0, 0, 0],
     [0, 0, 8, 0, 0, 0, 7, 2, 0],
     [0, 0, 0, 0, 8, 0, 0, 0, 0],
     [0, 3, 2, 5, 0, 0, 0, 0, 0],
     [0, 0, 0, 7, 0, 0, 0, 0, 3]]
    output:
    [[7 9 1 2 5 3 8 4 6]
     [8 4 5 6 9 7 2 3 1]
     [3 2 6 4 1 9 6 8 7]
     [1 7 3 8 2 4 5 9 6]
     [9 5 4 3 6 1 8 7 2]
     [6 2 8 9 7 5 7 2 4]
     [5 6 7 1 8 2 3 1 9]
     [4 3 2 5 9 6 1 7 8]
     [1 8 9 7 4 3 2 6 3]]





Algorytm dla 3x3 oraz 6x6 rozwiązuje problem poniżej 2/3. Natomiast dla 9x9 potrafi trwać nawet do kilku minut i nie zawsze wykona on go poprawnie. Dla 3x3 i 6x6 algorytm zawsze znajduje rozwiazanie, jesli istnieje. Przy wiekszych liczbach niestety jest problem.
